In [1]:
# Installing dependencies
! pip install pyspark pandas

In [9]:
! pyspark --packages org.apache.iceberg:iceberg-spark-runtime-3.2_2.12:0.13.1

Python 3.12.3 (main, Apr 17 2024, 00:00:00) [GCC 14.0.1 20240411 (Red Hat 14.0.1-0)] on linux
Type "help", "copyright", "credits" or "license" for more information.
:: loading settings :: url = jar:file:/home/ravik/src_git/LP/env/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/ravik/.ivy2/cache
The jars for the packages stored in: /home/ravik/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.2_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-0d0c6906-345f-4ec3-a80d-3a9a712755d6;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.2_2.12;0.13.1 in central
downloading https://repo1.maven.org/maven2/org/apache/iceberg/iceberg-spark-runtime-3.2_2.12/0.13.1/iceberg-spark-runtime-3.2_2.12-0.13.1.jar ...
	[SUCCESSFUL ] org.apache.iceberg#iceberg-spark-runtime-3.2_2.12;0.13.1!iceberg-spark-runtime-3.2_2.12.jar (11107ms)
:: resolution report ::

In [13]:
from pyspark.sql import SparkSession

# Initialize Spark session with Iceberg JAR
spark = SparkSession.builder \
    .appName("IcebergExample") \
    .config("spark.jars", "/home/ravik/src_git/jars/iceberg-spark-runtime-3.5_2.12-1.5.1.jar") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.spark_catalog.type", "hadoop") \
    .config("spark.sql.catalog.spark_catalog.warehouse", "/path/to/warehouse") \
    .getOrCreate()

# Create a DataFrame
data = [
    (1, 'Alice', 85, 90, 88, 92),
    (2, 'Bob', 78, 82, 80, 76),
    (3, 'Charlie', 95, 87, 91, 89),
    (4, 'David', 68, 74, 72, 70),
    (5, 'Eve', 88, 92, 84, 86)
]
columns = ['roll_num', 'name', 'sub1', 'sub2', 'sub3', 'sub4']
df = spark.createDataFrame(data, columns)

# Write DataFrame to Iceberg table
df.write.format("iceberg").mode("overwrite").save("spark_catalog.default.students")

# Perform updates
updated_data = [
    (1, 'Alice', 85, 90, 88, 95),  # Updated sub4 score
    (6, 'Frank', 90, 85, 87, 90)   # New student
]
updated_df = spark.createDataFrame(updated_data, schema=columns)
updated_df.createOrReplaceTempView("updates")

# Merge updates into the Iceberg table
spark.sql("""
MERGE INTO spark_catalog.default.students AS target
USING updates AS source
ON target.roll_num = source.roll_num
WHEN MATCHED THEN UPDATE SET *
WHEN NOT MATCHED THEN INSERT *
""")

# Query with time travel (adjust timestamp as needed)
timestamp = "2024-06-14T12:00:00.000Z"
df_time_travel = spark.read.format("iceberg").option("as-of-timestamp", timestamp).load("spark_catalog.default.students")
df_time_travel.show()

# Stop Spark session
spark.stop()


24/06/15 08:57:03 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


Py4JJavaError: An error occurred while calling o110.save.
: org.apache.spark.SparkClassNotFoundException: [DATA_SOURCE_NOT_FOUND] Failed to find the data source: iceberg. Please find packages at `https://spark.apache.org/third-party-projects.html`.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.dataSourceNotFoundError(QueryExecutionErrors.scala:724)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:647)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSourceV2(DataSource.scala:697)
	at org.apache.spark.sql.DataFrameWriter.lookupV2Provider(DataFrameWriter.scala:863)
	at org.apache.spark.sql.DataFrameWriter.saveInternal(DataFrameWriter.scala:257)
	at org.apache.spark.sql.DataFrameWriter.save(DataFrameWriter.scala:240)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:75)
	at java.base/jdk.in

In [12]:
import pyspark
pyspark.__version__

'3.5.1'